# Neural machine translation with attention

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
t = 0
e_X = []
b_Y = []
for line in open('/content/drive/My Drive/NLP/ben.txt'):
  # only keep a limited number of samples
  t += 1
  # input and target are separated by tab
  if '\t' not in line:
    continue
  # split up the input and translation
  input_text, translation, attrib = line.rstrip().split('\t')
  e_X.append( input_text )
  b_Y.append( translation )

In [ ]:
# Download the file
# path_to_zip = tf.keras.utils.get_file(
#     'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
#     extract=True)

#path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"
#path_to_file = "/content/dataset.xlsx"

In [ ]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  #w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  #w = re.sub(r"([?.!,¿])", r" \1 ", w)
  #w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  #w = re.sub(r"[^?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
en_sentence = u"May I borrow this book?"
#sp_sentence = u"¿Puedo tomar prestado este libro?"
sp_sentence = u"পচা মাছ কিনার কোনই দরকার নাই"

print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence))
print(sp_sentence)

<start> May I borrow this book? <end>
<start> পচা মাছ কিনার কোনই দরকার নাই <end>
পচা মাছ কিনার কোনই দরকার নাই


In [ ]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
  #lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  #word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
  #return zip(*word_pairs)
  
  # dataset = pd.read_excel(path_to_file)
  # dataset['Banglish']
  # return [preprocess_sentence(w) for w in tuple(dataset['Full Bangla'])] , [preprocess_sentence(w) for w in tuple(dataset['Banglish'])]
 
  return [preprocess_sentence(w) for w in tuple( b_Y )] , [preprocess_sentence(w) for w in tuple( e_X )]

In [ ]:
en, sp = create_dataset("", None)
print(en[-1])
print(sp[-1])
type(en)
import numpy as np
npp = np.array(sp)
npp.shape

<start> আমি ভেবেছিলাম এটা করা সহজ হবে, কিন্তু আমরা সারাদিন ধরে কাজ করেছি আর এখনো শেষ করে উঠতে পারিনি। <end>
<start> I thought doing this would be easy, but we've been working all day and we're still not finished. <end>


(4350,)

In [ ]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [ ]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [ ]:
# Try experimenting with the size of that dataset
num_examples = 4350
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.001)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

4345 4345 5 5


In [ ]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [ ]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
101 ----> ask
451 ----> again
392 ----> later.
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
107 ----> আবার
254 ----> পরে
78 ----> জিজ্ঞাসা
498 ----> করবে।
2 ----> <end>


In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 21]), TensorShape([64, 20]))

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    
    self.gru = tf.keras.layers.Bidirectional( tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform', dropout=0.5) )

  def call(self, x, hidden):
    x = self.embedding(x)
    output, f_h, f_c, b_h, b_c  = self.gru(x, initial_state = hidden)
    state_h = tf.keras.layers.Concatenate()([f_h, b_h])
    state_c = tf.keras.layers.Concatenate()([f_c, b_c])
    state = tf.convert_to_tensor([state_h, state_c])
    # state = tf.keras.layers.Concatenate()([state_h, state_c])
    return output, state_h, state

  def initialize_hidden_state(self):
    # return tf.zeros((self.batch_sz, self.enc_units))
    return [tf.zeros((self.batch_sz, self.enc_units)) for i in range(4)]

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
print( tf.convert_to_tensor( sample_hidden ).shape )
sample_output, sample_hidden, enc_state = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))
print( enc_state[0].shape )

(4, 64, 1024)
Encoder output shape: (batch size, sequence length, units) (64, 21, 2048)
Encoder Hidden state shape: (batch size, units) (64, 2048)
(64, 2048)


In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print( tf.convert_to_tensor( sample_output ).shape )
print( tf.convert_to_tensor( sample_hidden ).shape )

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

(64, 21, 2048)
(64, 2048)
Attention result shape: (batch size, units) (64, 2048)
Attention weights shape: (batch_size, sequence_length, 1) (64, 21, 1)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.LSTM(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform', dropout=0.5)
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output, enc_state):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state, _ = self.gru(x, initial_state = [enc_state[0], enc_state[1]] )

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units*2, BATCH_SIZE)

sample_decoder_output, state_decoder, attention_weights_decoder = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output, enc_state)

print( tf.convert_to_tensor(sample_decoder_output).shape )
print( tf.convert_to_tensor(state_decoder).shape )
print( tf.convert_to_tensor(attention_weights_decoder).shape )

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

(64, 3543)
(64, 2048)
(64, 21, 1)
Decoder output shape: (batch_size, vocab size) (64, 3543)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
# checkpoint_dir = './training_checkpoints'
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# checkpoint = tf.train.Checkpoint(optimizer=optimizer,
#                                  encoder=encoder,
#                                  decoder=decoder)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden, full_enc_state = encoder(inp, enc_hidden )

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output, full_enc_state)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 30

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  print( tf.convert_to_tensor(enc_hidden).shape )

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  #if (epoch + 1) % 2 == 0:
  #  checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

(4, 64, 1024)
Epoch 1 Batch 0 Loss 0.0327
Epoch 1 Loss 0.0474
Time taken for 1 epoch 34.56386923789978 sec

(4, 64, 1024)
Epoch 2 Batch 0 Loss 0.0364
Epoch 2 Loss 0.0476
Time taken for 1 epoch 34.63686275482178 sec

(4, 64, 1024)
Epoch 3 Batch 0 Loss 0.0521
Epoch 3 Loss 0.0488
Time taken for 1 epoch 34.313172578811646 sec

(4, 64, 1024)
Epoch 4 Batch 0 Loss 0.0408
Epoch 4 Loss 0.0488
Time taken for 1 epoch 34.71059322357178 sec

(4, 64, 1024)
Epoch 5 Batch 0 Loss 0.0438
Epoch 5 Loss 0.0477
Time taken for 1 epoch 34.376290798187256 sec

(4, 64, 1024)
Epoch 6 Batch 0 Loss 0.0345
Epoch 6 Loss 0.0471
Time taken for 1 epoch 34.52840518951416 sec

(4, 64, 1024)
Epoch 7 Batch 0 Loss 0.0454
Epoch 7 Loss 0.0483
Time taken for 1 epoch 34.587098598480225 sec

(4, 64, 1024)
Epoch 8 Batch 0 Loss 0.0384
Epoch 8 Loss 0.0480
Time taken for 1 epoch 34.52070093154907 sec

(4, 64, 1024)
Epoch 9 Batch 0 Loss 0.0418
Epoch 9 Loss 0.0490
Time taken for 1 epoch 34.5425386428833 sec

(4, 64, 1024)
Epoch 10 Bat

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate(u'i wish i can fly')
translate(u'i wish i can fly.')

<start> i wish i can fly <end>
Input: <start> i wish i can fly <end>
Predicted translation: আমি যদি সাঁতার কাটতে পারতাম। <end> 
<start> i wish i can fly. <end>
Input: <start> i wish i can fly. <end>
Predicted translation: আমি খালি কোথা থেকে পাব? <end> 


In [ ]:
translate(u"i do not like you.")

<start> i do not like you. <end>
Input: <start> i do not like you. <end>
Predicted translation: আমি আপনার কথা বুঝতে পারছি না। <end> 


In [ ]:
translate(u'i am eating')

<start> i am eating <end>
Input: <start> i am eating <end>
Predicted translation: আমি ফুটবল খেলি। <end> 


In [ ]:
#translate(u'esta es mi vida.')

In [ ]:
#translate(u'¿todavia estan en casa?')

In [ ]:
# wrong translation
#translate(u'trata de averiguarlo.')

In [ ]:
# value = ''
# while True:
#   value = input()
#   if value == 'quit':
#     break
#   translate(value.lower())

In [ ]:
how are you?
আপনি কেমন আছেন ?

i wish i can fly.
আমি যদি সাঁতার কাটতে পারতাম।

i like both.
আমার দুটোই পছন্দ।

i was very scared.
আমি খুব ভয়ে পেয়েগেছিলাম।

leave me alone.
আমাকে কেউ বোঝে না।

i don't like you.
আমি তোমাকে ভালো লাগে না।

In [ ]:
string_lists = [ 'how are you?', 'i wish i can fly', 'i like both.', 'i was very scared.', 'leave me alone.', 'i don\'t like you.' ]
for i in string_lists:
  print( translate( i.lower())  )

<start> how are you? <end>
Input: <start> how are you? <end>
Predicted translation: তুমি কেমন আছো? <end> 
None
<start> i wish i can fly <end>
Input: <start> i wish i can fly <end>
Predicted translation: আমি যদি সাঁতার কাটতে পারতাম। <end> 
None
<start> i like both. <end>
Input: <start> i like both. <end>
Predicted translation: আমার দুটোই পছন্দ। <end> 
None
<start> i was very scared. <end>
Input: <start> i was very scared. <end>
Predicted translation: আমি খুব ভয়ে পেয়েগেছিলাম। <end> 
None
<start> leave me alone. <end>
Input: <start> leave me alone. <end>
Predicted translation: আমাকে একলা ফেলে রেখে যাবেন না। <end> 
None
<start> i don't like you. <end>
Input: <start> i don't like you. <end>
Predicted translation: আমি আপনার বিশ্বাস করি। <end> 
None


In [ ]:
vai tui koi?
<start> vai tui koi ? <end>
Input: <start> vai tui koi ? <end>
Predicted translation: তুই কি পাগল হইয়া গেছেন ? <end> 
ami valo achi
<start> ami valo achi <end>
Input: <start> ami valo achi <end>
Predicted translation: আমি গেছিলাম যে ওদের বাসায় তখন <end> 
ami valo achi.
<start> ami valo achi . <end>
Input: <start> ami valo achi . <end>
Predicted translation: আমি আছি . <end> 
tui akhon koi?
<start> tui akhon koi ? <end>
Input: <start> tui akhon koi ? <end>
Predicted translation: তুই সন্ধ্যায় থাকবোনা বাসায় <end> 
tui koi?
<start> tui koi ? <end>
Input: <start> tui koi ? <end>
Predicted translation: তুই সন্ধ্যায় থাকবোনা বাসায় <end> 
tui vhai ay.
<start> tui vhai ay . <end>
Input: <start> tui vhai ay . <end>
Predicted translation: হা বাসায় তুই আই পেরা নাই <end> 
tui jokhon paros koris.
<start> tui jokhon paros koris . <end>
Input: <start> tui jokhon paros koris . <end>
Predicted translation: তুই পাইলে দিছ <end> 
Vai tui koi?
<start> vai tui koi ? <end>
Input: <start> vai tui koi ? <end>
Predicted translation: তুই কি পাগল হইয়া গেছেন ? <end> 
vai tui koi?
<start> vai tui koi ? <end>
Input: <start> vai tui koi ? <end>
Predicted translation: তুই কি পাগল হইয়া গেছেন ? <end> 
ami r jabo na.
<start> ami r jabo na . <end>
Input: <start> ami r jabo na . <end>
Predicted translation: আমি ডিসিশন নিচ্ছি আর থাকি না দিয়া করতে চাই <end> 
kemon jani.
<start> kemon jani . <end>
Input: <start> kemon jani . <end>
Predicted translation: খুব ভালো <end> 
valo onk.
<start> valo onk . <end>
Input: <start> valo onk . <end>
Predicted translation: হা বাসায় অনেক <end> 
valo onek.
<start> valo onek . <end>
Input: <start> valo onek . <end>
Predicted translation: হা বাসায় <end> 
vai onk valo.
<start> vai onk valo . <end>
Input: <start> vai onk valo . <end>
Predicted translation: ভাই জোস ক্লিক করছেন তো <end> 
apni kemon achen?
<start> apni kemon achen ? <end>

## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs
